In [1]:
! pip install geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv

/bin/bash: /media/mutakabbir/HDD_2TB_02/Forest_Fire/.venv/bin/pip: /media/mutakabbir/HDD_2TB_01/Forest_Fire/.venv/bin/python: bad interpreter: No such file or directory


In [2]:
from sqlalchemy import create_engine 
import geopandas as gpd
from shapely.ops import nearest_points

import os
from dotenv import load_dotenv

# Create Database Connection

In [3]:
PATH_TO_DOT_ENV = "../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

In [4]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

In [5]:
engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

## Database Constants

In [6]:
CWEEDS_META_TABLE_NAME = "W_m"
CWEEDS_META_SUBDIVISION_TABLE = "W_ms"
SUBDIVISION_TABLE_NAME = "S"

In [12]:
subdivision_query = f"""select s.cid, s.geometry from "{SUBDIVISION_TABLE_NAME}" s"""
cweeds_mata_query = f"""select wm."climate_ID", wm.prov, wm.geometry from "{CWEEDS_META_TABLE_NAME}" wm"""

# Read station and division data

In [8]:
division_df = gpd.read_postgis(
    sql=subdivision_query, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326"
)  
# division_ddf = dgpd.from_geopandas(division_df, npartitions=N_PARTISIONS)
# del division_df

In [14]:
cweeds_meta_df = gpd.read_postgis(
    sql=cweeds_mata_query, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326"
)  
# cweeds_meta_ddf = dgpd.from_geopandas(cweeds_meta_df, npartitions=N_PARTISIONS)
# del cweeds_meta_df

# Find all the staions in division

In [15]:
stations_in_subdivision = gpd.sjoin(
    cweeds_meta_df,
    division_df,
    how="inner",
    predicate="within").drop("index_right", axis=1)

In [16]:
stations_in_subdivision

,climate_ID,prov,geometry,cid
0,3010010,AB,POINT (-112.97000 54.28000),4
1,3010237,AB,POINT (-112.28000 53.92000),4
2,3060406,AB,POINT (-112.82000 54.78000),4
33,3012275,AB,POINT (-112.87000 53.68000),4
56,3013959,AB,POINT (-110.07000 53.31000),4
...,...,...,...,...
559,2100805,YT,POINT (-139.84000 67.57000),0
560,2100935,YT,POINT (-136.22000 66.98000),0
561,2101102,YT,POINT (-132.73000 60.17000),0
562,2101201,YT,POINT (-128.82000 60.12000),0


In [17]:
stations_in_subdivision[['cid','climate_ID']].to_sql(
    name=CWEEDS_META_SUBDIVISION_TABLE, 
    con=engine, 
    if_exists='replace', 
    index=False
)

510

In [18]:
stations_in_subdivision = stations_in_subdivision.groupby("cid").agg(lambda group: set(group))

In [19]:
# stations_in_subdivision.apply(lambda row: len(row.climate_ID), axis=1).plot(kind='hist',bins=35,figsize=(25, 5))

In [20]:
divisions_with_station = stations_in_subdivision.index.to_list()
print(f"stations found {len(divisions_with_station)}")

stations found 67


# Find the nearest station for division woth no stations

In [21]:
division_with_no_stations = division_df[~division_df["cid"].isin(divisions_with_station)]
division_with_no_stations

,cid,geometry


# if found divisions with no stations run below code 

In [15]:
def get_nearest_station_id(division):
    divison_id = division.FEDUID
    division_geomaetry = division.geometry
    _, nearest_point = nearest_points(division_geomaetry.centroid, cweeds_meta_df.geometry.unary_union)
    station_id = cweeds_meta_df.loc[cweeds_meta_df['geometry'] == nearest_point].climate_ID.values[0]
    return station_id

In [16]:
# division_with_no_stations['climate_ID'] = division_with_no_stations.apply(get_nearest_station_id, axis=1)

In [17]:
# division_with_no_stations[['FEDUID','climate_ID']].to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False)

# Add Primary Key to Table

In [19]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE "{CWEEDS_META_SUBDIVISION_TABLE}" ADD PRIMARY KEY ("cid", "climate_ID");')